<a href="https://colab.research.google.com/github/kokami236/kozinkadai/blob/master/tengunkiriwake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================
# 0. ライブラリのインストール
# =========================
!pip install open3d -q

# =========================
# 1. 点群ファイルをアップロード
# =========================
from google.colab import files
import os

upload = files.upload()  # ローカルから .ply を1個アップする
file_name = list(upload.keys())[0]  # 最初の1個だけ使う

print("uploaded:", file_name)

# =========================
# 2. 点群の読み込み
# =========================
import open3d as o3d
import numpy as np

pcd = o3d.io.read_point_cloud(file_name)
print(pcd)

# 点数を確認
points = np.asarray(pcd.points)
print("original points:", points.shape)

# =========================
# 3. 簡単な前処理（任意で調整）
#    - 統計的アウトライヤ除去
#    - ボクセルダウンサンプル
# =========================

# (1) アウトライヤ除去
# nb_neighbors と std_ratio は点群によって変える
pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=30, std_ratio=2.0)
print("after outlier removal:", np.asarray(pcd.points).shape)

# (2) ボクセルダウンサンプル
# 城なら 0.02〜0.05 あたりが現実的。まずは0.03m。
voxel_size = 0.03
pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
print("after voxel downsample:", np.asarray(pcd_down.points).shape)

# 法線もこの段階でつけておくと後で楽
pcd_down.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30)
)

# numpy化
pts = np.asarray(pcd_down.points)        # (N, 3)
nrm = np.asarray(pcd_down.normals)       # (N, 3)

# =========================
# 4. XYでタイル分割
#    ここでは「城全体を等間隔グリッドに割る」単純版
#    tile_size を変えると細かさが変わる
# =========================

tile_size = 5.0  # 5m x 5m のタイルにする
margin_z = 0.5   # z方向を少し広めにとると段差があっても抜けにくい

min_xyz = pts.min(axis=0)
max_xyz = pts.max(axis=0)

min_x, min_y, min_z = min_xyz
max_x, max_y, max_z = max_xyz

# 何タイルになるかざっくり計算
nx = int(np.ceil((max_x - min_x) / tile_size))
ny = int(np.ceil((max_y - min_y) / tile_size))
print(f"will create tiles: {nx} x {ny}")

os.makedirs("tiles_npz", exist_ok=True)

tile_count = 0
for ix in range(nx):
    for iy in range(ny):
        x0 = min_x + ix * tile_size
        x1 = x0 + tile_size
        y0 = min_y + iy * tile_size
        y1 = y0 + tile_size

        # このタイルに入る点を抽出
        mask = (
            (pts[:, 0] >= x0) & (pts[:, 0] < x1) &
            (pts[:, 1] >= y0) & (pts[:, 1] < y1)
        )
        tile_pts = pts[mask]
        tile_nrm = nrm[mask]

        if tile_pts.shape[0] == 0:
            continue  # 点がないタイルはスキップ

        # 必要ならz方向の余白をつけたい人はここにさらに条件を足す
        # 今回はxyだけで切ってる

        # npzで保存
        out_path = f"tiles_npz/tile_x{ix:02d}_y{iy:02d}.npz"
        np.savez_compressed(
            out_path,
            xyz=tile_pts.astype(np.float32),
            normal=tile_nrm.astype(np.float32),
            bbox=np.array([[x0, y0, min_z - margin_z], [x1, y1, max_z + margin_z]], dtype=np.float32)
        )
        tile_count += 1

print("saved tiles:", tile_count)

# =========================
# 5. Colabにまとめてダウンロードしたい場合
# =========================
import shutil
shutil.make_archive("tiles_npz", "zip", "tiles_npz")

files.download("tiles_npz.zip")
